# Normalización de activaciones con normalización de capas

Entrenar  redes  neuronales  profundas  con  múltiples  capas  puede resultar  a  veces  complicado  debido  a  problemas  como  la  desaparición  o  la  explosión  de  gradientes.  Estos  problemas  generan  dinámicas  de  entrenamiento  inestables  y  dificultan  que  la  red  ajuste  eficazmente  sus  pesos,  lo  que  implica  que  el  proceso  de  aprendizaje  tiene dificultades  para  encontrar  un  conjunto  de  parámetros  (pesos)  que  minimice  la  función  de  pérdida.  En  otras  palabras,  la  red  tiene  dificultades  para  aprender  los  patrones  subyacentes  en  los  datos  a  un  nivel  que  le  permita  realizar  predicciones  o  tomar  decisiones  precisas.

En estas sección se implementará   la  normalización  de  capas  para  mejorar  la  estabilidad  y Eficiencia  del  entrenamiento  de  redes  neuronales.

La  idea  principal  detrás  de  la  normalización  de  capas  es  ajustar  las  activaciones  (salidas)  de  una  capa  de  red  neuronal  para  tener  una  media  de  0  y  una  varianza  de  1,  también  conocida  como  varianza  unitaria.
Este  ajuste  acelera  la  convergencia  a  pesos  efectivos  y  garantiza  un  entrenamiento  consistente  y  fiable.  

Descripción general visual de cómo funciona la normalización de capas.

![Texto alternativo](./imgs/4.5.png)

Se puede recrear el ejemplo recreado.


In [ ]:
import torch
import torch.nn as nn

torch.manual_seed(123)
batch_example = torch.randn(2, 5) #dos ejemplos de cinco características
layer = nn.Sequential(nn.Linear(5, 6), nn.ReLU())
output = layer(batch_example)
print(output)

tensor([[0.2260, 0.3470, 0.0000, 0.2216, 0.0000, 0.0000],
        [0.2133, 0.2394, 0.0000, 0.5198, 0.3297, 0.0000]],
       grad_fn=<ReluBackward0>)


La  capa  de  red  neuronal  que  hemos  codificado  consta  de  una  capa  lineal  seguida  de  una  función  de  activación  no  lineal,  ReLU  (abreviatura  de  Unidad  Lineal  Rectificada),  que  es  una  función  de  activación  estándar  en  redes  neuronales.  

Antes de aplicar la normalización de capas a las salidas, hay que examinar la media y la diferencia.

In [9]:
mean = output.mean(dim=-1, keepdim=True) #alcula la media a lo largo de la última dimensión del tensor output
var = output.var(dim=-1, keepdim=True)
print("Media: ", mean)
print("Var: ", var)
output.shape

Media:  tensor([[0.1324],
        [0.2170]], grad_fn=<MeanBackward1>)
Var:  tensor([[0.0231],
        [0.0398]], grad_fn=<VarBackward0>)


torch.Size([2, 6])

La  primera  fila  del  tensor  de  media  anterior  contiene  el  valor  medio  de  la  primera  fila  de  entrada,  y  la  
segunda  fila  de  salida  contiene  la  media  de  la  segunda  fila  de  entrada.
El  uso  de  keepdim=True  en  operaciones  como  el  cálculo  de  la  media  o  la  varianza  garantiza  que  el  tensor  de  salida  conserve  el  mismo  número  de  dimensiones  que  el  tensor  de  entrada,  incluso  si  la  
operación  reduce  el  tensor  según  la  dimensión  especificada  mediante  dim.  
Por  ejemplo,  sin  keepdim=True,  el  tensor  de  la  media  devuelto  sería  un  vector  bidimensional  [0,1324,0,2170]  en  lugar  de  una  matriz  de  2×1  dimensiones  [[0,1324],  [0,2170]]. El  parámetro  dim  especifica  la  dimensión  a  lo  largo  de  la  cual  se  realiza  el  cálculo  de  la  estadística (aquí,  media  o  varianza)  debe  realizarse  en  forma  de  tensor.

![Texto alternativo](./imgs/4.6.png)

Al  añadir  la  normalización  de  capas  al  modelo  GPT,  que  genera  tensores  3D  con  forma  [batch_size,  num_tokens,  embedding_size],  podemos  seguir  usando  dim=1  para  la  normalización  en  la  última  dimensión,  evitando  así  un  cambio  de  dim=1  a  dim=2.

A  continuación,  se aplicará la  normalización  de  capas  a  las  salidas  de  capa  obtenidas  anteriormente.  
La  operación  consiste  en  restar  la  media  y  dividirla  entre  la  raíz  cuadrada  de  la  varianza  (también  
conocida  como  desviación  estándar)

In [10]:
out_norm = (output - mean) / torch.sqrt(var)
mean = out_norm.mean(dim=-1, keepdim=True)
var = out_norm.var(dim=-1, keepdim=True)
print("Normalized layer outputs:\n", out_norm)
print("Mean:\n", mean)
print("Variance:\n", var)

Normalized layer outputs:
 tensor([[ 0.6159,  1.4126, -0.8719,  0.5872, -0.8719, -0.8719],
        [-0.0189,  0.1121, -1.0876,  1.5173,  0.5647, -1.0876]],
       grad_fn=<DivBackward0>)
Mean:
 tensor([[9.9341e-09],
        [0.0000e+00]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


In [11]:
#Para mejorar la visualizació  se puede elimianr la notación cientiífica
torch.set_printoptions(sci_mode=False)
print("Mean:\n", mean)
print("Variance:\n", var)

Mean:
 tensor([[    0.0000],
        [    0.0000]], grad_fn=<MeanBackward1>)
Variance:
 tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


Hasta  ahora,  en  esta  sección,  hemos  codificado  y  aplicado  la normalización  de  capas  paso  a  paso.  Ahora,  encapsulemos  este  proceso  en  un  módulo  de  PyTorch  que  podremos  usar  posteriormente  en  el  modelo  GPT.